## 爬取目标

<div align=center>
    <img src="../../../images/fb0cc800f904f30931b46f0af6dd6c257e0a709d67c0dec84766a0f3562b9be4.png" width=75%>
    </img>
</div>

- 1. 提取每一页的电影列表,以及每个电影的详情页

- 2. 提取每部电影的`名称`、`封面`、`类别`、`上映时间`、`评分`、`剧情简介`

- 3. 存入数据库

- 4. 多进程

### 1. 提取每一页的电影列表,以及每个电影的详情页

In [2]:
import requests
import re
from pyquery import PyQuery as pq

url = 'https://ssr1.scrape.center'

res = requests.get(url)
res

<Response [200]>

In [44]:
# 将获取的HTML文本存储至文件中

with open('11基础案例HTML.html', 'w') as fp:
    fp.write(res.text)

#### 使用 css 选择器

In [3]:
def get_page_info(res):
    
    movies_page = re.findall('<li.*?number.*?>.*?<a href="(.*?)">(.*?)</a>.*?</li>', res.text, re.S)
    
    page_infos = []
    for page in movies_page:
        page_info = {
            'page_name':page[1],
            'page_url':'https://ssr1.scrape.center' + page[0]
        }
        page_infos.append(page_info)
    return page_infos

a = get_page_info(res)

In [91]:
b = a[0]
b['page_url']

'https://ssr1.scrape.center/page/1'

In [9]:
import re
result = re.search('<div.*?p-h el-col el-col-24 el-col-xs-9 el-col-sm-13 el-col-md-16">.*?<a.*?href="(.*?)".*?>.*?</a>', res.text, re.S)
print(result.group())

<div id="app">
  <div data-v-74e8b908="" class="el-row" id="header">
    <div data-v-74e8b908="" class="container el-col el-col-18 el-col-offset-3">
      <div data-v-74e8b908="" class="el-row">
        <div data-v-74e8b908="" class="logo el-col el-col-4">
          <a data-v-74e8b908="" href="/" class="router-link-exact-active router-link-active">
            <img data-v-74e8b908="" src="/static/img/logo.png" class="logo-image">
            <span data-v-74e8b908="" class="logo-title">Scrape</span>
          </a>
        </div>
      </div>
    </div>
  </div>
  
<div data-v-7f856186="" id="index">
  <div data-v-7f856186="" class="el-row">
    <div data-v-7f856186="" class="el-col el-col-18 el-col-offset-3">
      
      <div data-v-7f856186="" class="el-card item m-t is-hover-shadow">
        <div class="el-card__body">
          <div data-v-7f856186="" class="el-row">
            <div data-v-7f856186="" class="el-col el-col-24 el-col-xs-8 el-col-sm-6 el-col-md-4">
              <a da

In [19]:
import re
result = re.search(, res.text, re.S)



'<div.*?index">'



print(result.group(1))

/detail/1"
                 class="">
                <img
                    data-v-7f856186=""
                    src="https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c"
                    class="cover">
              </a>
            </div>
            <div data-v-7f856186="" class="p-h el-col el-col-24 el-col-xs-9 el-col-sm-13 el-col-md-16">
              <a data-v-7f856186="" href="/detail/1
